In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import numpy as np
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows', 1000)

In [3]:
# Run individually with 100M, 150M, 200M, 250M, 300M, 400M
DOLLAR_MIN = 200000000

In [4]:
STABLE_COINS   = ['USDT', 'USDC']
UNSTABLE_COINS = ['BTC', 'ETH']
JOIN_WINDOW_MINUTES = 30 
PLOT_SPAGHETTI = True

spaghetti_suffix = '_spaghetti' if PLOT_SPAGHETTI else ''
dollar_suffix = int(DOLLAR_MIN / 1e6)
PLOT_FILE_NAME = f'average_price_impact_{dollar_suffix}M{spaghetti_suffix}.html'

In [5]:
def prep_prices_df(prices_df):
    """
    Fiter to just unstable coins (BTC/ETH), and rename columns
    """        
    prices_df = prices_df.rename(columns={
        'mid': 'price', 
        'unix_time': 'price_time', 
        'asset': 'price_asset'
    })
    
    prices_df = prices_df[['price_asset', 'price', 'price_time']]
    
    return prices_df

def prep_tweets_df(tweets_df, dollar_min):
    """
    Round tweet to nearest minute and filter for only transactions about {dollar_min} dollars
    """
    def _round_to_minute(tweet_time):
        sec_after_min = tweet_time % 60
        return tweet_time - sec_after_min if sec_after_min <= 30 else tweet_time - sec_after_min + 60
    
    tweets_df['tweet_time'] = tweets_df['unix_time'].apply(_round_to_minute)
    
    tweets_df = tweets_df[(tweets_df['usd_value'] >= dollar_min)]
    
    tweets_df = tweets_df.rename(columns={'asset': 'tweet_asset'})
    
    return tweets_df

In [6]:
def subset_tweets(tweets_df, transfer_type, coin_type, source, recipient):
    """
    Filters the twitter dataframe to the given transfer type
    """
            
    is_stable_coin   = tweets_df['tweet_asset'].isin(STABLE_COINS)
    is_unstable_coin = tweets_df['tweet_asset'].isin(UNSTABLE_COINS)
    
    coin_cond = is_stable_coin if coin_type == 'stable' else is_unstable_coin

    transfer_type_col = tweets_df['transaction_type']
    source_col        = tweets_df['transaction_source']
    recipient_col     = tweets_df['transaction_recipient']
    
    # If it was a mint, all we need to filter for is the transaction type
    if transfer_type == 'MINTED':
        return tweets_df[(is_stable_coin) & (transfer_type_col == 'MINTED')]
    
    # Otherwise, filter on coin type, transfer type, source and recipient
    else:
        filter_stmt = (
            (coin_cond) 
            & (transfer_type_col == transfer_type) 
            & (source_col == source) 
            & (recipient_col == recipient)
        ) 
        return tweets_df[filter_stmt]

In [7]:
def join_tweets_and_prices(tweets_df, prices_df, coin_type):
    """
    Joins prices within {JOIN_WINDOW_MINUTES} of the tweet
    """
    
    # We'll take the cartesian product for the join and then filter
    df = tweets_df.merge(prices_df, how='cross')

    # For the plots from unstable coin tweets, 
    #  we'll only want to plot the price change for the corresponding coin
    #  i.e. we only want to plot BTC price changes from BTC tweets
    #
    # For plots about stable coins, we'll plot both BTC and ETH
    if coin_type == 'unstable':
        df = df[(df['tweet_asset'] == df['price_asset'])]

    # Filter to just prices within the specified time window of the tweet 
    time_filter = (
        (df['price_time'] >= (df['tweet_time'] - (JOIN_WINDOW_MINUTES * 60)))
        & (df['price_time'] <= (df['tweet_time'] + (JOIN_WINDOW_MINUTES * 60)))
    )

    df = df[time_filter]
        
    return df

In [8]:
def get_price_changes(df):
    """
    Get price change at each minute interval surrounding the tweet
    """
    # Get time difference between tweet and price in minutes
    df['time_delta'] = ((df['price_time'] - df['tweet_time']) / 60).astype('int')
    
    # Get a dataframe with just the starting prices
    start_prices_df = df[df['time_delta'] == 0]
    start_prices_df = start_prices_df.rename(columns={'price': 'start_price'})
    start_prices_df = start_prices_df[['price_asset', 'start_price', 'tweet_time']]
    
    # Join the t=0 price back to the original dataframe
    df = df.merge(start_prices_df, how='inner', on=['tweet_time', 'price_asset'])
    
    # Calculate percent price change
    df['price_change'] = ((df['price'] - df['start_price']) / df['start_price']) * 100
        
    return df

def get_average_price_change(df):
    """
    Get the weighted average price change
    Weighted by transaction size in dollars
    """
    return (df
            .groupby(['price_asset', 'time_delta'])
            .apply(lambda df: np.average(df['price_change'], weights=df['usd_value']))
            .reset_index(name='price_change'))

In [9]:
def get_price_plots(average_price_change_df, all_price_change_df, title, num_tweets, plot_spaghetti):
    """
    Returns a plot of the average price change for both BTC and ETH surrounding a whale alert
    Input dataframe has 1 record per asset/time_delta combination
    """
    dark_color = 'rgb(57,118,175)'
    light_color = 'rgba(57,118,175, 0.2)'
    
    figs = []
        
    for asset in UNSTABLE_COINS:
        
        num_tweets_asset = num_tweets[asset] if isinstance(num_tweets, dict) else num_tweets
        
        asset_avg_df = average_price_change_df[average_price_change_df['price_asset'] == asset]
        asset_all_df = all_price_change_df[all_price_change_df['price_asset'] == asset]
                        
        rolling_window = 4
        rolling_name = f'{rolling_window}_min_rolling_average'
        
        asset_avg_df = asset_avg_df.assign(
            rolling=asset_avg_df['price_change'].rolling(rolling_window).mean().shift(-1 * (rolling_window // 2))
        )
        asset_avg_df = asset_avg_df.rename(columns={'rolling': rolling_name})
        
        fig = None
        if plot_spaghetti:
            asset_avg_df = asset_avg_df[['time_delta', rolling_name]]
            
            asset_all_df = asset_all_df.pivot_table(index='time_delta', columns='tweet_time', values='price_change')
            asset_all_df = asset_all_df.reset_index()
            tweet_times = [c for c in asset_all_df.columns if c != 'time_delta']

            asset_all_df = asset_all_df.merge(asset_avg_df, on='time_delta', how='inner')
            
            fig = px.line(asset_all_df, 
                          x='time_delta', 
                          y=[rolling_name, *tweet_times] , 
                          color_discrete_sequence=[dark_color, *[light_color for _ in tweet_times]],
                          title=title.format(asset, num_tweets_asset))
            
            fig.update_layout(showlegend=False)

        else:
            fig = px.line(asset_avg_df, 
                          x='time_delta', 
                          y=['price_change', rolling_name] , 
                          color_discrete_sequence=[light_color, dark_color],
                          title=title.format(asset, num_tweets_asset))
            
        fig.add_vline(x=0, line_color='lightgrey', line_width=3)
        fig.add_hline(y=0, line_color='lightgrey', line_width=3)
        
        fig.update_layout(yaxis_range=[-1, 1])
        fig.update_yaxes(tickvals=list(np.linspace(-1, 1, 5)))
        
        fig.update_layout(height=400, width=1200)
        
        figs.append(fig)
    
    return figs

In [10]:
def get_plot_title(transfer_type, tweet_coin_type, source, recipient):

    def _get_non_mint_event():
        coins = STABLE_COINS if tweet_coin_type == 'stable' else UNSTABLE_COINS
        coins_str = '/'.join(coins)
        
        return f'Transfer of <b>{coins_str}</b> from <b>{source.capitalize()}</b> to <b>{recipient.capitalize()}</b>'

    event = 'USDC/USDT Minting' if transfer_type == 'MINTED' else _get_non_mint_event()
        
    title = f'Weighted Price Change of <b>{{}}</b> surrounding {event} <br>'
    title += f'Composed of {{}} tweets'
    
    return title
    

In [13]:
def get_random_tweet_df(num_tweets, lower_bound, upper_bound):
    """
    Creates a dummy tweets dataframe with random times to test the typical trend
    """    
    # Add a buffer so there's enough room for prices
    # Divide by 60 and remove decimal so there are minute increments
    start_minute = (lower_bound + 7200) // 60
    end_minutes = (upper_bound - 7200) // 60
    
    return pd.DataFrame({
        'tweet_time': [random.randint(start_minute, end_minutes) * 60 for _ in range(num_tweets)],
        'usd_value': [1 for _ in range(num_tweets)]
    })

| Direction          |     Option Name    | Event                                        |
|--------------------|:-------------------|:---------------------------------------------|
| **Minting**        |       `mint`       | USDT/USDC Minting                            |
| **Bullish**        | `mint_to_exchange` | USDT/USDC Transfer from treasury to exchange |
| **Bullish**        |    `sell_stable`   | USDT/USDC Transfer from wallet to exchange   |
| **Bearish**        | `sell_unstable`    | BTC/ETH Transfer from wallet to exchange     |

In [12]:
prices_df = pd.read_csv('../data/prices_formatted.csv')
tweets_df = pd.read_csv('../data/tweets_formatted.csv')

tweets_df = prep_tweets_df(tweets_df, DOLLAR_MIN)
prices_df = prep_prices_df(prices_df)

## Event Plots

In [96]:
plots = [
#     ('Minting',                               'MINTED',   'stable',   None,        None),
#     ('Bullish - Stable Treasury to Exchange', 'TRANSFER', 'stable',   'TREASURY', 'EXCHANGE'),
    ('Bullish - Stable Wallet to Exchange',   'TRANSFER', 'stable',   'WALLET',   'EXCHANGE'),
    ('Bearish - BTC/ETH Wallet to Exchange',  'TRANSFER', 'unstable', 'WALLET',   'EXCHANGE'),
]

plot_file = f'../plots/{PLOT_FILE_NAME}'        
html_file = open(plot_file, 'w')

heading = f'Weighted Average Price Change Surrounding Whale Alerts - ${DOLLAR_MIN:,} Minimum'
html_file.write(f'<h1 style="font-family:arial">{heading}<h1>')

figs = []
for (direction, transfer_type, tweet_coin_type, source, recipient) in plots:
    
    html_file.write(f'<h2 style="font-family:arial">{direction}<h2>')
    
    example_tweet_df = subset_tweets(tweets_df, transfer_type, tweet_coin_type, source, recipient)
    
    # Get the number of tweets - if plotting stable coins, we take the total number of tweets
    # If plotting unstable coins, split out the count for each coin
    num_tweets = example_tweet_df.shape[0]
    if tweet_coin_type == 'unstable':
        num_tweets = {
            coin: example_tweet_df[example_tweet_df.tweet_asset == coin].shape[0]
            for coin in UNSTABLE_COINS
        }
    
    joined_df = join_tweets_and_prices(example_tweet_df, prices_df, tweet_coin_type)
    all_price_change_df = get_price_changes(joined_df)
    
    average_price_change_df = get_average_price_change(all_price_change_df)
        
    title = get_plot_title(transfer_type, tweet_coin_type, source, recipient)
    
    [html_file.write(fig.to_html()) 
     for fig in get_price_plots(average_price_change_df, all_price_change_df, title, num_tweets, PLOT_SPAGHETTI)]
     
html_file.close()

## Baseline Plots

In [17]:
for num_fake_tweets in [20, 50, 100]:
    plot_file = f'../plots/average_price_impact_baseline_{num_fake_tweets}_tweets.html'
    html_file = open(plot_file, 'w')

    heading = f'Weighted Average Price Change Surrounding {num_fake_tweets} Random Points in Time'
    html_file.write(f'<h1 style="font-family:arial">{heading}<h1>')

    lower_bound_time = prices_df['price_time'].min()
    upper_bound_time = prices_df['price_time'].max()
    random_tweet_df = get_random_tweet_df(num_fake_tweets, lower_bound_time, upper_bound_time)

    joined_df = join_tweets_and_prices(random_tweet_df, prices_df, 'stable')

    all_price_change_df = get_price_changes(joined_df)
    average_price_change_df = get_average_price_change(all_price_change_df)

    title = f'Price Change of <b>{{}}</b>'

    [html_file.write(fig.to_html()) 
     for fig in get_price_plots(average_price_change_df, all_price_change_df, title, num_fake_tweets, False)]

    html_file.close()

In [19]:
# Confirm baseline was uniform
px.scatter(random_tweet_df.assign(y=0), x='tweet_time', y='y')